In [2]:
#importing pandas,sqland numpy
import pandas as pd
import numpy as np
import sqlite3 

In [ ]:
#Step 1: Load CSV Data and viewing of data 
order= pd.read_csv("orders.csv")
print(order.head())
print(order.tail())
print(order.describe())


   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                  restaurant_name  
0               New Foods Chinese  
1  Ruchi Curry House Multicuisine  
2           Spice Kitchen Punjabi  
3          Darbar Kitchen Non-Veg  
4       Royal Eatery South Indian  
      order_id  user_id  restaurant_id  order_date  total_amount  \
9995      9996     2528            249  21-05-2023       1211.96   
9996      9997     2867            267  06-08-2023       1188.05   
9997      9998      522            420  11-11-2023        979.44   
9998      9999      319            492  08-09-2023       1105.93   
9999     10000      457            439  21-10-2023   

In [7]:
#Step 2: Load JSON Data and checking the data
user = pd.read_json("users.json")
print(user.head())
print(user.info())
print(user.describe())


   user_id    name       city membership
0        1  User_1    Chennai    Regular
1        2  User_2       Pune       Gold
2        3  User_3  Bangalore       Gold
3        4  User_4  Bangalore    Regular
4        5  User_5       Pune       Gold
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     3000 non-null   int64 
 1   name        3000 non-null   object
 2   city        3000 non-null   object
 3   membership  3000 non-null   object
dtypes: int64(1), object(3)
memory usage: 93.9+ KB
None
           user_id
count  3000.000000
mean   1500.500000
std     866.169729
min       1.000000
25%     750.750000
50%    1500.500000
75%    2250.250000
max    3000.000000


In [8]:
#first thing we should connect our sql db to localfile with extension(.db) in our local storage(in vs code)
con = sqlite3.connect("restaurants.db")
#and next step is to execute read operation for reading entire 
with open("restaurants.sql", "r") as f:
    sql_script = f.read()
con.executescript(sql_script)
#Step 3: Load SQL Data
restaurant = pd.read_sql(
    "SELECT * FROM restaurants",
    con
)
print(restaurant.head())
print(restaurant.info())
print(restaurant.describe())



   restaurant_id restaurant_name  cuisine  rating
0              1    Restaurant_1  Chinese     4.8
1              2    Restaurant_2   Indian     4.1
2              3    Restaurant_3  Mexican     4.3
3              4    Restaurant_4  Chinese     4.1
4              5    Restaurant_5  Chinese     4.8
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   restaurant_id    500 non-null    int64  
 1   restaurant_name  500 non-null    object 
 2   cuisine          500 non-null    object 
 3   rating           500 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 15.8+ KB
None
       restaurant_id      rating
count     500.000000  500.000000
mean      250.500000    4.040600
std       144.481833    0.604469
min         1.000000    3.000000
25%       125.750000    3.500000
50%       250.500000    4.100000
75%       375.250000 

In [13]:
#Step 4: Merge the Data using some major join methods like natural join,left join, right join etc.
orderuser = order.merge(
    user,
    on="user_id",
    how="left"
)
# using left join for previoiusly mergerd orderuser data and resturants data
finaldata = orderuser.merge(
    restaurant,
    on="restaurant_id",
    how="left"
)


In [14]:
#final step is saving the finaldata file in my local storage 
finaldata.to_csv("final_food_delivery_dataset.csv")


In [40]:
# for answering mcq questions i will use group_by etc methods 
d= pd.read_csv("final_food_delivery_dataset.csv")
#1
g = d[d["membership"] == "Gold"]
c = g.groupby("city")["total_amount"].sum()
print(c.sort_values(ascending=False))
print()
#2
av = d.groupby("cuisine")["total_amount"].mean()
print(av.sort_values(ascending=False))
print()
#3
us= d.groupby("user_id")["total_amount"].sum()
cus = len(us[us > 1000])
print(cus)
print()
#4
d["rating_range"] = pd.cut(
    d["rating"],
    [3.0, 3.5, 4.0, 4.5, 5.0],
    labels=["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]
)
r = d.groupby("rating_range")["total_amount"].sum()
print(r.sort_values(ascending=False))
print()
#5
g= d.groupby("city")["total_amount"].mean()
print(g.sort_values(ascending=False))
print()
#6
rt= d.groupby("cuisine")["restaurant_id"].nunique()
rev = d.groupby("cuisine")["total_amount"].sum()
result = pd.DataFrame({
    "restaurant_count": rt,
    "revenue": rev
})
print(result.sort_values(by=["restaurant_count", "revenue"],
                          ascending=[True, False]))
print()
#7
t = len(d)
gd = len(d[d["membership"] == "Gold"])
p = round((gd/t)*100)
print(p)
print()
#8
d["restaurant_name"]=d["restaurant_name_x"].combine_first(d["restaurant_name_y"])
rd= d.groupby("restaurant_name_x").agg(
    orders=("order_id", "count"),
    avg_value=("total_amount", "mean")
)
f= rd[rd["orders"] <=20]
print(rd.sort_values("avg_value", ascending=False))
print()
#9
com = d.groupby(
    ["membership", "cuisine"]
)["total_amount"].sum()
print(com.sort_values(ascending=False))
print()
#10
d["order_date"] = pd.to_datetime(d["order_date"])
d["quarter"] = d["order_date"].dt.quarter
q = d.groupby("quarter")["total_amount"].sum()
print(q.sort_values(ascending=False))




city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

2544

rating_range
4.6–5.0    2197030.75
4.1–4.5    1960326.26
3.0–3.5    1881754.57
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

city
Chennai      806.202118
Hyderabad    803.985779
Bangalore    802.234308
Pune         792.097914
Name: total_amount, dtype: float64

         restaurant_count     revenue
cuisine                              
Chinese               120  1930504.65
Italian               126  2024203.80
Indian                126  1971412.58
Mexican               128  2085503.09

50

                            orders    avg_value
restaurant_name_x                              
Classic Biryani Chinese         26  1126.520000
Hotel Dhaba Multicuisine        13  1040.222308
Sri Mess Punjabi          

C:\Users\Sai kumar\AppData\Local\Temp\ipykernel_12544\3231895831.py:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  r = d.groupby("rating_range")["total_amount"].sum()
C:\Users\Sai kumar\AppData\Local\Temp\ipykernel_12544\3231895831.py:62: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  d["order_date"] = pd.to_datetime(d["order_date"])


In [48]:
#FIB section
#1
gr= d[d["membership"] == "Gold"]
print(len(gr))
#2
h = d[d["city"] == "Hyderabad"]["total_amount"].sum()
print(round(h))
#3
un=d["user_id"].nunique()
print(un)
#4
gl = d[d["membership"] == "Gold"]["total_amount"].mean()
print(round(gl, 2))
#5
hg=d[d["rating"]>=4.5]
print(len(hg))
#6
ct = gr.groupby("city")["total_amount"].sum()
ty = ct.idxmax()
ok= gr[gr["city"] == ty]
print(len(ok))
print(len(d))


4987
1889367
2883
797.15
3374
1337
10000
